# Runke Kutta with Linear Programming - Fallback approach

The main idea in this notebook is to integrate an ode with an Runge Kutta method. If an runge Kutta step does't lead to an positive result the b is changed in a way that the it enshures positifity. The method with the new b has one order less than the original method

This needs a RK with embeded methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()


ck5 =rk.loadRKM('CK5').__num__()
dp5 =rk.loadRKM('DP5').__num__()


In [ ]:
def tau(k,A,c): 
    #generates tau vector 
    return 1/ np.math.factorial(k)*c**k - 1/np.math.factorial(k-1) *A @ c**(k-1)
    
def tau_hat(k,A,c):
    return c**k-k*A@(k-1)


def OrderCond(A,c,order = 1):
    #Generates Order Condition Matrix O and right side vector r for Linear Equation System O@b=r
    
    s = len(c) #number of stages
    
    r = []
    O_rows = []
    
    
    if A.shape != (s,s):
        raise InputError
        
    else:
        if order >= 1:
            O_rows.append(np.ones(s));      r.append(1)
            
        if order >= 2:
            O_rows.append(c);               r.append(1/2)
            
        if order >= 3:
            O_rows.append(c**2);            r.append(1/3)
            O_rows.append(tau(2,A,c));      r.append(0.)
            
        if order >= 4:
            O_rows.append(c**3);            r.append(1/4)
            O_rows.append(tau(2,A,c)*c);    r.append(0.)
            O_rows.append(tau(2,A,c)@A.T);  r.append(0.)
            O_rows.append(tau(3,A,c));      r.append(0.)
        
        if order >= 5:
            O_rows.append(c**4);                     r.append(1/5)
            O_rows.append(A@np.diag(c)@tau(2,A,c));  r.append(0.)
            O_rows.append(A@A@tau(2,A,c));           r.append(0.)
            O_rows.append(A@tau(3,A,c));             r.append(0.)
            O_rows.append(tau(4,A,c));               r.append(0.)
            O_rows.append(np.diag(c)@A@tau(2,A,c));  r.append(0.)
            O_rows.append(np.diag(c)@tau(3,A,c));    r.append(0.)
            O_rows.append(np.diag(c**2)@tau(2,A,c)); r.append(0.)
            O_rows.append(tau(2,A,c)**2);            r.append(0.)
        if order >= 6:
            print('too high order')
            raise NotImplementedError
        
        O = np.vstack(O_rows)
        return (O,np.array(r))
            
                


In [ ]:
ck5.p

In [ ]:
#Test Function
rkm = ck5
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p)

print(O@rkm.b-rhs)

print ('Size of O:')
print (O.shape)

print('Rank')
print(linalg.matrix_rank(O))


In [ ]:
O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p-1)
print ('Size of O:')
print (O.shape)

print('Rank')
print(linalg.matrix_rank(O))

In [ ]:
def RK_variable_b(rkm, dt, f, w0=[1.,0], t_final=1.,b_fixed = False,solver = cp.ECOS):
    """    
    Options:
    
        rkm: Base Runge-Kutta method, in Nodepy format
        dt: time step size
        f: RHS of ODE system
        w0: Initial data
        t_final: final solution time        
    """
    
    #setup Variables for Soulution storage
    p = len(w0) #number of dimentions
    
    uu = np.zeros([p,int(t_final/dt)+100])
    uu[:,0] = w0.copy()
    tt = np.zeros([int(t_final/dt)+100])
    
    
    #Setup Runge Kutta 
    c = rkm.c
    A = rkm.A #has to be lower left triangle
    s = len(c) #number of Stages
    K = np.zeros([p,s])
    
    u = np.array(w0)
    t = 0.
    n = 0
    
    
    #Setup Optimisation Problem
    O, rhs = OrderCond(rkm.A,rkm.c,order = rkm.p-1)
    ap_op =cp.Variable(s)
    an_op =cp.Variable(s)
    e = np.ones(s) #vector for gola Fnction, just generates the 1-Norm of b
    
    
          #Maybee set up Problem here and treat H as an Paramter
        
    #for debugging b's    
    bb = np.zeros([s,int(t_final/dt)+2])
        
    #print('set up starting to solve')
    
    #Solve ODE
    while t<t_final:
        for i in range(s):
            #compute Stages
            
                
            #K[:,i] = f(t+c[i]*dt,u+dt*K@A[i,:]) 
            #the 0s in A should make shure that no data from an older Step is used
            
            #Maybe better Approach, because A[i,j] = 0 in many places
            u_prime = u.copy()
            for m in range(i):
                u_prime += dt*A[i,m]*K[:,m]
            
            K[:,i] = f(t+c[i]*dt,u_prime)
            
            #print('intermediatestep computed')
        
        if b_fixed == False:
            #test if positifity is correct
            if (u+dt*K@rkm.b >= 0).all():
                b =rkm.b
            
            else:
            #Run Optimisation Problem
        
                prob = cp.Problem(cp.Minimize(e@ap_op+e@an_op),
                              [O@(ap_op-an_op+rkm.b)==rhs,u+dt*K@(ap_op-an_op+rkm.b)>=0,ap_op>=0,an_op>=0])  
                prob.solve(solver=solver)
                if prob.status != cp.OPTIMAL:
                    print(prob.status)
            
                b = ap_op.value - an_op.value + rkm.b
                print('b changed')
        else:
            b =rkm.b
        #update
        u += dt*K@b
        n += 1
        t += dt
        
        uu[:,n] = u.copy()
        bb[:,n] = b.copy()
        tt[n] = t
        #print('updated')

        
    return (tt[0:n],uu[:,0:n],bb[:,0:n])
        
        
    



In [ ]:
ck5.plot_stability_region()

# Testproblems



## Testproblem Time integrator

In [ ]:
def f_sin(t,u):
    return -np.sin(t)

def f_cos(t,u):
    return np.cos(t)

def f_const(t,u):
    return 1

u0 =np.array([2.0])

t,u,b = RK_variable_b(ck5,0.4,f_sin,w0=u0,t_final=100,b_fixed=False)


In [ ]:
plt.plot(t,u[0,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

## Testproblem Harmonic Oscilator

In [ ]:
def f_A(t,u):
    A = np.array([[0,-1],[1,0]])
    c = np.array([1,1])
    #print(u)
    return A@(u-c)

u0 =np.array([1.,0.])

t,u,b = RK_variable_b(ck5,0.2,f_A,w0=u0,t_final=200,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

## Testproblem from Kopecz and Meister 2018

In [ ]:
def f_lin_I(t,u):
    a = 5 # a>0
    A = np.array([[-a,1],[a,-1]])
    return A@u

u0 =np.array([0.9,0.1])

t,u,b = RK_variable_b(ck5,0.6,f_lin_I,w0=u0,t_final=100,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

In [ ]:
#Nonlinear test problem
def f_nonlin(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = -(u[0]*u[1])/(u[0]+1)
    du[1] = (u[0]*u[1])/(u[0]+1) -a*u[1]
    du[2] = a*u[1]
    return du

u0 = np.array([9.98,0.01,0.01])

t,u,b = RK_variable_b(ck5,2,f_nonlin,w0=u0,t_final=30,b_fixed=False)



In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])

plt.grid()

In [ ]:
plt.plot(t[1:],b.T[1:,:])

In [ ]:
# Brusselator test problem
def f_brusselator(t,u):
    k1 =1
    k2 =1
    k3 =1
    k4 =1
    k5 =1
    k6 =1
    du = np.zeros(6)
    du[0] = -k1*u[0]
    du[1] = -k2*u[1]*u[4]
    du[2] = k3*u[1]*u[4]
    du[3] = k4*u[4]
    du[4] = k1*u[0] - k2*u[1]*u[4] + k3*u[4]**2*u[5] - k4*u[4]
    du[5] = k2*u[1]*u[4] - k3*u[4]**2*u[5]
    return du

u0 = np.array([10.,10.,0.,0.,0.1,0.1])


t,u,b = RK_variable_b(ck5,0.3,f_brusselator,w0=u0,t_final=6,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.plot(t,u[1,:])
plt.plot(t,u[2,:])
plt.plot(t,u[3,:])
plt.plot(t,u[4,:])
plt.plot(t,u[5,:])

In [ ]:
plt.plot(t,b.T)

In [ ]:
#Robertson test problem, stiff
def f_robertson(t,u):
    a = 0.3
    du = np.zeros(3)
    du[0] = 1e4 *u[1]*u[2] - 0.04*u[0]
    du[1] = 0.04 *u[0] - 1e4*u[1]*u[2] - 3e7*u[1]**2
    du[2] = 3e7*u[1]**2
    return du

u0 = np.array([1.,0.,0.])

#t,u,b = RK_variable_b(ssp104,1e-6,f_robertson,w0=u0,t_final=1,b_fixed=True)

# Decay with Production

ODE of form $u'(t) = - \lambda u(t) + f(t)$ with $f(t) \geq 0 $ 

In [ ]:
def production(t):
    return 0.05*(1+np.sin(t))

def f_decay(t,u):
    l = 1
    return -l*u+production(t)

u0 = np.array([1.])

t,u,b = RK_variable_b(ck5,3,f_decay,w0=u0,t_final=200,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.grid()

In [ ]:
plt.plot(t[1:],b.T[1:,:])

# Testproblem


In [ ]:
def f_test(t,u):
    return np.sin(10*t)*u*(1-u)

u0 = np.array([0.1])

t,u,b = RK_variable_b(dp5,1.1,f_test,w0=u0,t_final=50,b_fixed=False)

In [ ]:
plt.plot(t,u[0,:])
plt.grid()

# Advection

In [ ]:
N=50
x = np.linspace(0,1,N)
dx = x[1]-x[0]

def f_centered_advection(t,u):
    du = np.zeros_like(u)
    du[1:-1] = (u[2:]-u[:-2])
    du[0] = u[1]-u[-1]
    du[-1] = u[0]-u[-2]
    return -du/(2*dx)

def f_upwind_advection(t,u):
    du = np.zeros_like(u)
    du[1:] = (u[1:]-u[:-1])
    du[0] = u[0]-u[-1]
    return -du/(dx)

#u0 = (x<0.5)*1.
#u0 = (x<0.5)+0.2
u0 = (x<0.5)+0.
#u0 = np.sin(2*np.pi*x)+1.

dt = 2.5*dx/3

t,u,b = RK_variable_b(ck5,dt,f_upwind_advection,w0=u0,t_final=1,b_fixed=False)

In [ ]:
plt.pcolor(u[:,::1])
plt.colorbar()

In [ ]:
plt.plot(u[:,5])

In [ ]:
plt.plot(t[1:],b.T[1:,:])

Most of the testproblems show stability issues before getting negative. 

Needed: A method that is stable in a great region but does not preserve positifity

Idea:

Other Conditions that can be writen as a linear Equation system $Au = r$ or $Au \geq r$ with $A$ Matrix of Condition and $r$ as right Hand side (That is not preserved by the nature of the runge Kutta Method) 
(as:  $A(Kb+u) = r$)  with $K= [k^1,\cdots k^{s-1}]$ where $k^m$ are the intermediate solutions computet by the stages 



Or dynamic right hand side? some $r(t,u-1)$  (thinking of Lyapunov functions)